In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from collections import Counter
import math, json, io
import distutils.dir_util

from keras.layers import Input, Dense
from keras.models import Model

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kakao-save-data/tag_song_dict_no0.json
/kaggle/input/kakao-save-data/tag_song_dict.json
/kaggle/input/kakao-save-data/__notebook__.ipynb
/kaggle/input/kakao-save-data/custom.css
/kaggle/input/kakao-save-data/song_tag_dict.json
/kaggle/input/kakao-save-data/__results__.html
/kaggle/input/kakao-save-data/__output__.json
/kaggle/input/kakao-save-data/__results___files/__results___15_0.png
/kaggle/input/hs-1-preprocess/__notebook__.ipynb
/kaggle/input/hs-1-preprocess/custom.css
/kaggle/input/hs-1-preprocess/__results__.html
/kaggle/input/hs-1-preprocess/__output__.json
/kaggle/input/hs-1-preprocess/train_DF.parquet
/kaggle/input/hs-1-preprocess/__results___files/__results___9_0.png
/kaggle/input/hs-1-preprocess/__results___files/__results___10_0.png
/kaggle/input/hs-1-preprocess/__results___files/__results___7_0.png
/kaggle/input/hs-1-preprocess/__results___files/__results___11_0.png
/kaggle/input/kakao-arena-melon/genre_gn_all.json
/kaggle/input/kakao-arena-melon/val.json
/k

Using TensorFlow backend.


In [2]:
def load_json(fname):
    with open(fname, encoding="utf-8") as f:
        json_obj = json.load(f)
    return json_obj

def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError
    parent = os.path.dirname(fname)
    distutils.dir_util.mkpath(parent)
    with io.open(fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

In [3]:
kaggle_path = '/kaggle/input/kakao-arena-melon/'

### genre_gn_all
# genre_gn_all = pd.read_json(kaggle_path+'genre_gn_all.json', typ = 'series')
# genre_gn_all = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})

### song_meta
song_meta = pd.read_json(kaggle_path+'song_meta.json', typ = 'frame')

### modeling_data
train = pd.read_json(kaggle_path+'train.json', typ = 'frame')
val = pd.read_json(kaggle_path+'val.json', typ = 'frame')
# test = pd.read_json(kaggle_path+'test.json', typ = 'frame')

train_pv = pd.read_parquet('/kaggle/input/hs-1-preprocess/train_DF.parquet')
song_tag_dict = load_json('/kaggle/input/kakao-save-data/song_tag_dict.json')

In [4]:
# index2id = {i:u for i, u in enumerate((train_pv.index))}
# id2index = {u:i for i, u in enumerate((train_pv.index))}
# # train_pv.reset_index(inplace = True, drop = True)

In [5]:
train_pv.head()

id,85,107,164,217,228,307,325,347,545,577,...,153005,153010,153051,153070,153074,153075,153187,153242,153280,153333
song,,,,,,,,,,,,,,,,,,,,,
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
print(train_pv.shape)
train_pv.T.sum()

(11065, 2703)


song
10         6.0
62        11.0
71        14.0
93         5.0
195        7.0
          ... 
707724    25.0
707768     7.0
707837    17.0
707849     5.0
707873     6.0
Length: 11065, dtype: float64

In [7]:
input_dim = train_pv.shape[1]

In [8]:
a = 1000
b = 500

input_img = Input(shape=(input_dim,)) ## input layer
encoded = Dense(a, activation='relu')(input_img) ## encoding dim으로 압축
encoded = Dense(b, activation='relu')(encoded) ## encoding dim으로 압축


decoded = Dense(b, activation='sigmoid')(encoded) ## input dim으로 복귀
decoded = Dense(input_dim, activation='sigmoid')(encoded) ## input dim으로 복귀

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

# create a placeholder for an encoded (32-dimensional) input01
# encoded_input = Input(shape=(encoding_dim,))
encoded_input = Input(shape=(b,))

# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

# compile
autoencoder.compile(optimizer='adam', loss='mse') # (optimizer='adadelta', loss='binary_crossentropy')

## train
tmp = train_pv.values
history = autoencoder.fit(tmp, tmp,
                epochs=5,
                batch_size=256,
                shuffle=True)

Epoch 1/5
44/44 [==============================] - 3s 73ms/step - loss: 0.0573
Epoch 2/5
44/44 [==============================] - 3s 64ms/step - loss: 0.0037
Epoch 3/5
44/44 [==============================] - 3s 65ms/step - loss: 0.0037
Epoch 4/5
44/44 [==============================] - 3s 64ms/step - loss: 0.0037
Epoch 5/5
44/44 [==============================] - 3s 63ms/step - loss: 0.0037


In [9]:
encoded_imgs = encoder.predict(tmp)
decoded_imgs = decoder.predict(encoded_imgs)

In [10]:
train_data = pd.DataFrame(data=encoded_imgs)

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_array = cosine_similarity(train_data, train_data)

### Predict

In [12]:
def most_popular(playlists, col, topk_count):
    c = Counter()

    for doc in playlists[col]:
        c.update(doc)

    topk = c.most_common(topk_count)
    return c, [k for k, v in topk]

def remove_seen(seen, l):
    seen = set(seen)
    ## 데이터에 l의 원소가 없으면 l의 원소를 채워 넣는다. 
    return [x for x in l if not (x in seen)]

def check_target_type(df):
    no_song_idx = []
    no_tag_idx  = []
    no_both_idx = []
    yes_all_idx = []

    for i in df.index:
        ## 노래 X 태그 O
        if   bool(not df.loc[i:i, 'songs'].values[0]) & bool(df.loc[i:i, 'tags'].values[0]):
            no_song_idx.append(i)
        ## 노래 O 태그 X
        elif bool(df.loc[i:i, 'songs'].values[0])     & bool(not df.loc[i:i, 'tags'].values[0]):
            no_tag_idx.append(i)  
        ## 노래 X 태그 X
        elif bool(not df.loc[i:i, 'songs'].values[0]) & bool(not df.loc[i:i, 'tags'].values[0]):
            no_both_idx.append(i)          
        ## 노래 O 태그 O (노래 or 태그 일부 누락)
        else:
            yes_all_idx.append(i)
            
    print("노래 X 태그 O : {}개".format(len(no_song_idx)))
    print("노래 O 태그 X : {}개".format(len(no_tag_idx)))
    print("노래 X 태그 X : {}개".format(len(no_both_idx)))
    print("노래 O 태그 O : {}개".format(len(yes_all_idx)))
    
    return no_song_idx, no_tag_idx, no_both_idx, yes_all_idx

In [13]:
no_song_idx, no_tag_idx, no_both_idx, yes_all_idx = check_target_type(val)

노래 X 태그 O : 2630개
노래 O 태그 X : 9661개
노래 X 태그 X : 1749개
노래 O 태그 O : 8975개


In [14]:
train['year'] = train['updt_date'].apply(lambda x: int(x[2:4]))
val['year'] = val['updt_date'].apply(lambda x: int(x[2:4]))

year_list = list(train['year'].unique())
year_list = sorted(year_list)

val3 = val[val.index.isin(yes_all_idx)]

percent=0.975
like_try = train['like_cnt'].quantile(percent)

# train_tmp = train[train['like_cnt'] > like_try]
val_tmp = val3[val3['like_cnt'] > like_try]
# val_tmp = val3[val3['year'].isin(small_years)]
# print("val year shape", val_tmp.shape)
# val_tmp.reset_index(inplace = True, drop = True)

In [15]:
popular_year = []
for i in year_list:
    tmp = train[train['year'] == i]
    _, year_song = most_popular(tmp, "songs", 200)
    _, year_tag = most_popular(tmp, "tags", 100)
    popular_year.append({
        "year" : i,
        "songs" : year_song,
        "tags" : year_tag,})

In [16]:
val3_predict = []
val_tmp = val3.copy()
for i in val_tmp.index:
    year = val_tmp.loc[i]['year']

    ################ song 채우기 ###############################
    songs = val_tmp.loc[i]['songs']
    songs = [s for s in songs if s in train_pv.index]
    if len(songs):
        len_song = math.ceil(round(200/len(songs),2))
        cur_song = []
        for s in songs:
            s_idx = train_pv.index.tolist().index(s)
            to_d = list(np.argsort(-cosine_array[s_idx])[1:1+len_song])            
#             to_update = [index2id[x] for x in to_d]
            cur_song = remove_seen(cur_song, to_d)
        cur_song = cur_song[:200]

    else:
        ## 부족하면 채워준다.
        cur_song = popular_year[year-4]['songs']

    cur_song = remove_seen(list(val_tmp.loc[i:i, 'songs'])[0], cur_song)[:100]
    
    #중복 케이스 때문에 부족하면 popular song으로 다시 채워줌
    if len(cur_song) != 100:
        update_song = remove_seen(cur_song, popular_year[year-4]['songs'])
        cur_song.extend(update_song)
        cur_song = cur_song[:200]
    
    
    ############### tag 채우기 ##################################
    tag_counter = Counter()
    for song in list(val_tmp.loc[i:i, 'songs'].values[0]):
        if str(song) in song_tag_dict:
            for tag in song_tag_dict[str(song)]:
                tag_counter.update({tag : 1})
    tag_counter = sorted(tag_counter.items(), key= lambda x:x[1], reverse = True)        
    tag_counter = tag_counter[:100] ## 넉넉히 100개 잡는다. 
    
    cur_tag = []
    for k in tag_counter:
        cur_tag.append(k[0])     
        
    if (len(cur_tag) < 100) & (len(cur_tag) > 0 ):
        ## 부족하면 채워준다.
        update_tag = remove_seen(cur_tag, popular_year[year-4]['tags'])
        cur_tag.extend(update_tag)
        cur_tag = cur_tag[:100]
    if len(cur_tag) == 0:
        cur_tag = popular_year[year-4]['tags'][:100]
        
    #############################################################    

    val3_predict.append({
        "id" : list(val_tmp.loc[i:i, 'id'])[0],
        "songs": remove_seen(list(val_tmp.loc[i:i, 'songs'])[0], cur_song)[:100],
        "tags": remove_seen(list(val_tmp.loc[i:i, 'tags'])[0], cur_tag)[:10],
     })    

In [17]:
len(val3_predict)

8975

In [18]:
write_json(val3_predict, 'val3_predict_with_like.json')